In [1]:
import os
print(os.getcwd())

/Users/maria/PycharmProjects/oxo_dG


In [2]:
from extractor.reader import FuzzySearch, SequenceMatch, FastqProcessor
from extractor.fuzzy_search import FuzzySearch
from extractor.fastq_extractor import extract_regions
from extractor import fastq_fragments_extractor
import pandas as pd
from typing import List

In [3]:
fastq_file = "sample_generation/synthetic_reads.fastq"

In [4]:
artificial_sequences = {
    "A1": "GATCAGTCCGATATC",
    "A2": "TCGACATGCTAGTGC",
    "A3": "GCTATCGGATACGTC",
    "S1": "ATGACTGCCA",
    "S2": "TGGCAGTCAT",
    "A1C": "GATATCGGACTGATC",
    "A2C": "GCACTAGCATGACGA",
    "A3C": "GACGTATCCGATAGC"
}

In [4]:
# reference = """
# TCGTGCTAAGCTTTAGGGCCNNNNNNNNTGCNGGCCCNNNNGCNNNGCNNNNGCTCTCGNNGCANNCGAGAGCNGGATCCNGCTCTCGNNTGCNNCGAGAGCNNNNGCNNNGCNNNNGGGCCNGCANNNNNNNNGGCCCNAAAGCTTAGCACGA
# """

In [5]:
# artificial_sequences = {
#     "S1": "TCGTGCTAAGCTTTAGGGCC",
#     "S2": "CGAGAGCAGGATCC",
#     "S3": "GGCCCTAAAGCTTAGCACGA"
# }

In [5]:
processor = FastqProcessor(fastq_file, artificial_sequences)
sequence_matches = processor.process_file()

Processing FASTQ: 100%|██████████| 1000/1000 [00:05<00:00, 173.38it/s]


In [6]:
sequence_matches

[SequenceMatch(read_id='synthetic_read_0', query_name='A1', position=0, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='A2', position=24, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='A3', position=48, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='S1', position=63, length=10, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='S2', position=79, length=10, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='A3C', position=89, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='A2C', position=113, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_0', query_name='A1C', position=137, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_1', query_name='A1', position=0, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_read_1', query_name='A2', position=24, length=15, score=0.9),
 SequenceMatch(read_id='synthetic_rea

In [7]:
processor.save_sequence_matches_to_csv(sequence_matches, 'service_files/sequence_matches.csv')

Saved 8000 matches to service_files/sequence_matches.csv


,read_id,query_name,position,length,score
0,synthetic_read_0,A1,0,15,0.9
1,synthetic_read_0,A2,24,15,0.9
2,synthetic_read_0,A3,48,15,0.9
3,synthetic_read_0,S1,63,10,0.9
4,synthetic_read_0,S2,79,10,0.9
...,...,...,...,...,...
7995,synthetic_read_999,S1,63,10,0.9
7996,synthetic_read_999,S2,79,10,0.9
7997,synthetic_read_999,A3C,89,15,0.9
7998,synthetic_read_999,A2C,113,15,0.9


In [8]:
def dataframe_to_sequence_matches(df: pd.DataFrame) -> List[SequenceMatch]:
    """
    Convert a pandas DataFrame back to a list of SequenceMatch objects.
    
    Assumes DataFrame has columns: ['read_id', 'query_name', 'position', 'length', 'score']
    """
    matches: List[SequenceMatch] = []

    for _, row in df.iterrows():
        matches.append(
            SequenceMatch(
                read_id=row['read_id'],
                query_name=row['query_name'],
                position=int(row['position']),
                length=int(row['length']),
                score=float(row['score'])
            )
        )
    return matches

In [9]:
sequence_matches_df = pd.read_csv('service_files/sequence_matches.csv')
sequence_matches = dataframe_to_sequence_matches(sequence_matches_df)

In [3]:
# Пути к данным
fastq_path = "sample_generation/synthetic_reads.fastq"
table_path = "service_files/sequence_matches.csv"

# Извлекаем участки
results = extract_regions(fastq_path, table_path)

# Посмотрим первые 3 результата
for r in results[:3]:
    print(r)

{'read_id': 'synthetic_read_0', 'Guanine': 'GACGGAGTC', 'Guanine_start:': 15, 'Guanine_end': 24, 'oxoGuanine': 'CATCGATAC', 'oxoGuanine_start': 39, 'oxoGuanine_end': 48, 'realGuanine': 'GCAGGCTGA', 'realGuanine_start': 128, 'realGuanine_end': 137, 'realOxoGuanine': 'CTACGCATA', 'realOxoGuanine_start': 104, 'realOxoGuanine_end': 113}
{'read_id': 'synthetic_read_1', 'Guanine': 'ACGGGGGGG', 'Guanine_start:': 15, 'Guanine_end': 24, 'oxoGuanine': 'GTCGGACTG', 'oxoGuanine_start': 39, 'oxoGuanine_end': 48, 'realGuanine': 'GGCAGGGGG', 'realGuanine_start': 128, 'realGuanine_end': 137, 'realOxoGuanine': 'GCTGGGTCA', 'realOxoGuanine_start': 104, 'realOxoGuanine_end': 113}
{'read_id': 'synthetic_read_10', 'Guanine': 'TGCGGTTTG', 'Guanine_start:': 15, 'Guanine_end': 24, 'oxoGuanine': 'ATCCGGATA', 'oxoGuanine_start': 39, 'oxoGuanine_end': 48, 'realGuanine': 'GCGTGGTTT', 'realGuanine_start': 128, 'realGuanine_end': 137, 'realOxoGuanine': 'CCTAGATAG', 'realOxoGuanine_start': 104, 'realOxoGuanine_end':

In [4]:
pd.DataFrame(results).to_csv('service_files/extracted_fastq_fragments.csv')

In [9]:
train_fragments = fastq_fragments_extractor.extract_dG_oxo_dG_fragments(fastq_file,
                                      sequence_matches)

In [10]:
train_fragments

[{'read_id': 'b5aaedc3-ac68-4529-8493-d3a72415caf6',
  'dG': 'TCGTGCTGAGCTTTAGGGC',
  'dG_start': 44,
  'dG_end': 63,
  'oxo_dG': 'CGAGAGCAGGATCCATGAAGATCGTGCTCTC',
  'oxo_dG_start': 63,
  'oxo_dG_end': 94}]